In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
%matplotlib inline 
# Sets the backend of matplotlib to 'inline' backend.
from matplotlib import pyplot as plt # plotting
from matplotlib import style # prepackaged plotting styles

import seaborn as sns #prettier and potentially more informative plotting than matplotlib

import string
import warnings
warnings.filterwarnings('ignore')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
 #   for filename in filenames:
  #      print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

 Understand nature of the data .info() .describe()
 Check for missing/null data
 Get value counts for categorical variables
 Data visualization using histograms, bar charts, and box plots 
 Correlation between the metrices 
 Explore probabilities:- 
    (1)Ladies and children better survival? 
    (2) Very young and elderly better survival?
    (3) having a family advantageous?
    (4) checking via expensive ticket - i.e. Pclass and Fare -  an  indicator of better survival odds?
    (5) More likely to survive with fancy title like Doctor?
    
 Feature engineering 
 preprocess data together or use a transformer? 
    # use label for train and test   
 Scaling/
 Baseline model check 
 comparison with varying CV 

In [ ]:
train = pd.read_csv('../input/titanic/train.csv')
test = pd.read_csv('../input/titanic/test.csv')
gs = pd.read_csv('../input/titanic/gender_submission.csv')

Visualising the column names, non-null counts, and data types for each variable in the training data set.

In [ ]:
train.info()

A quick summary :=
in total 891 entries
some null values in the Age--177null, Cabin---687null, and Embarked columns---2 nulls.

quick change , a conversion of age presently in float is to convert to integer as all ages is not a  decimal function

cabin null can be a feature update to the likelihood of survival of the passengers
thereby can create category as null_cabin can be helpful for the predictions.

Now,Examining the first few rows of training data

In [ ]:
train.head()

Discovering the column names, non-null counts, and data types for each variable in the training data set


In [ ]:
test.info()

summary--There are 418 records in the test dataframe, with null values in Age (86), Fare (1), and Cabin (327)

Examing the first few rows of test data

In [ ]:
test.head()

till now we didn't have any information regarding whether the passenger survived or not !!

So Now, Examining the training dataset numeric and categorical variables;

In [ ]:
train.describe(include='all')

Now the main part comes i.e. *Visualizing the data*
Firstly, let's examine the relationship between Sex and Survived. Our hypothesis(also in accordance with the movie) is that women and children were more likely to survive than men, but was this statistacally true?




**Womwn and Men Survived overall percentage** :--

In [ ]:
women = train.loc[train.Sex == 'female']['Survived']
rate_women = sum(women)/len(women)*100
print("% of women who survived:", round(rate_women, 2), "%")

men = train.loc[train.Sex == 'male']['Survived']
rate_men = sum(men)/len(men)*100
print("% of men who survived:", round(rate_men, 2), "%")

plotting the data into a pie chart representation 

In [ ]:
male = train[train['Sex']=='male'].groupby('Survived').count()['Sex']
female = train[train['Sex']=='female'].groupby('Survived').count()['Sex']

#create pie charts for men and women
pie_colors = ['tab:orange', 'tab:blue']
labels = ['died', 'survived']

fig, axs = plt.subplots(nrows=1, ncols=2,figsize=(10, 4))
fig.suptitle('Survival Percentages by Sex', fontsize=16)

axs[0].pie(x=male, autopct="%.1f%%", labels=['died', 'survived'], colors=pie_colors, wedgeprops={'edgecolor':'black', 'linewidth':1})
axs[0].set_title('Males')

axs[1].set_title('Females')
axs[1].pie(x=female, autopct="%.1f%%", labels=['died', 'survived'], colors=pie_colors, wedgeprops={'edgecolor':'black', 'linewidth':1})

plt.tight_layout()
plt.show()

similarly , using a barchart !!

In [ ]:
counts = train.groupby('Sex').Survived.value_counts()
counts

In [ ]:
gender_survival = pd.DataFrame({'Sex': ['female', 'male'], 'Survived': [counts[0], counts[3]], 'Died': [counts[1], counts[2]]})
gender_survival.plot(x='Sex', kind='bar', rot=0, title='Gender-->', ylabel='Count-->', edgecolor='black', linewidth=1)
plt.tight_layout()
plt.show()

# Analysing the actual presence of men and women overall on the ship using bar chart 

In [ ]:
train['Sex'].value_counts()

In [ ]:
train['Sex'].value_counts().plot(kind='bar', rot=0, edgecolor='black', linewidth=1.5, color=['C0', 'C1'])

plt.title('Number of Males and Females Aboard', fontsize=16)
plt.ylabel('Total Passengers-->')
plt.xlabel('Gender-->', fontsize=10)

plt.tight_layout()
plt.show()

# #A quick analysis :-
as the ratio of survival of women were more than the survival of men beside being in the scenario that number of men is more than women . hence becoming a very strong indicator of whether or not a passenger was to survive the Titanic sinking.

******now,histogram to compare the survival rates across gender and age 

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=2,figsize=(10, 4)) #plotting age on x axis and count on y axis
fig.suptitle('Survival Histograms by Sex', fontsize=16)

ax1 = sns.histplot(data=train[train['Sex']=='male'], x='Age', hue='Survived', binwidth=5, binrange=(0,80), ax=axs[0])
ax2 = sns.histplot(data=train[train['Sex']=='female'], x='Age', hue='Survived', binwidth=5, binrange=(0,80), ax=axs[1])
ax1.set_title('Males')
ax2.set_title('Females')

for ax in [ax1, ax2]:
    ax.legend(labels=['survived', 'died'])

    plt.tight_layout()
plt.show()

hence,
Males between the ages of 15 and 75 were unlikely to survive. Females in all age groups were more likely to survive than perish, but females between the ages of 5 and 15 had relatively higher mortality rates. Let's find out exactly what the rate was for young females

In [ ]:
young_females = train[(train['Sex']=='female') & (train['Age']>=5.0) & (train['Age']<=15.0)]
young_females.describe()

checking for young males between age of 5 and 15

In [ ]:
young_males = train[(train['Sex']=='male') & (train['Age']>=5.0) & (train['Age']<=15.0)]
young_males.describe()

Females between ages 5 and 15 inclusive had a 61 percent chance of survival. Males in the same age group had a 35 percent chance of survival.


Data Modification :-
**To enhance the dataset, we'll fill in the NaN(not any number) values for Age, Fare, Cabin, and Embarked fields !**

In [ ]:
train2 = train.copy()
test2 = test.copy()

df_corr = train2.corr().unstack().sort_values(kind='quicksort', ascending=False).reset_index()
df_corr.rename(columns={'level_0':'Feature 1', 'level_1': 'Feature 2', 0: 'Correlation'}, inplace=True)
df_corr[df_corr['Feature 1'] == 'Age']


The strongest positive and negative correlations for Age are with log_fare (0.102) and with Pclass (-.326). SibSp and Parch are also relatively strongly correlated at -.224 and -.168 respectively



Now, data cleaning on the basis of passenger's class or pclass feature!
And hence plotting it via bar chart.

In [ ]:
age_via_pclass = train2.groupby(['Sex', 'Pclass'])['Age'].median()
age_via_pclass

In [ ]:
ax = plt.subplot()
age_via_pclass.plot(kind='bar', rot=0, edgecolor='black', linewidth=1, color=['tab:orange', 'tab:blue', 'tab:green'])
plt.ylabel('Age', fontsize=12)
plt.xlabel('Passenger Class-->', fontsize=12)
ax.set_xticklabels(['First, F', 'Second, F', 'Third, F', 'First, M', 'Second, M', 'Third, M'])
plt.tight_layout()
plt.show()

We can see that first class passengers were oldest, followed by second class, then third class, with males slightly older in each category, can use this information to impute the ages for the passengers with missing age values.

In [ ]:
train2.isnull().sum()

In [ ]:
train2.groupby(['Sex', 'Pclass'])['Age'].median()

In [ ]:
train2.groupby(['Sex', 'Pclass'])['Fare'].median()

filling up of NaN data(Age,Fare) by just taking the median of values present 

In [ ]:
data = [train2, test2]
for dataset in data:
    dataset['Age'] = dataset.groupby(['Sex', 'Pclass'])['Age'].apply(lambda x: x.fillna(x.median()))
    dataset['Fare'] = dataset.groupby(['Sex', 'Pclass'])['Fare'].apply(lambda x: x.fillna(x.median()))

Arbitrary Check 

In [ ]:
train2.isnull().sum()

In [ ]:
test2.isnull().sum()

The null values for Age are now removed from the new training and test data frames, and have been replaced with the median ages for each Sex/Pclass combination.



now,separating out numeric and categorical variables

In [ ]:
train_quant = train.select_dtypes(np.number)
train_quant.drop(['PassengerId'], axis=1, inplace=True)
train_cat = train.select_dtypes('object')
print(train_quant, train_cat)

In [ ]:
train_quant.corr()

Age distribution:-
The histogram below indicates that the age variable follows a relatively normal distribution, with most passengers in in the 20-40 year range.

In [ ]:
plt.hist(train.Age, bins=8, linewidth=1, edgecolor='red')
plt.title('Passenger_Age_Distribution')
plt.xlabel('Age-->')
plt.ylabel('Count-->')
plt.show()

* **Fare**
The Fare distribution is heavily right skewed, indicating a small number of passengers purchasing very expensive tickets. Using a log transformation, we can normalize this variable.

In [ ]:
plt.hist(train.Fare, bins=20, linewidth=1.2, edgecolor='black')
plt.title("Ticket Fare Distribution")
plt.ylabel("Count")
plt.xlabel("Fare in Dollars")
plt.show()

In [ ]:
train['log_fare'] = np.log(train['Fare']+1)
test['log_fare'] = np.log(test['Fare']+1)
plt.hist(train.log_fare, bins=20, linewidth=1, edgecolor='black', color='C4')
plt.title("Log Ticket Fare Distribution")
plt.ylabel("Count")
plt.xlabel("Log Fare")
plt.show()

In [ ]:
train2['log_fare'] = np.log(train2['Fare']+1)
test2['log_fare'] = np.log(test2['Fare']+1)

In [ ]:
train2['Title'] = train2['Name'].apply(lambda x: x.split(', ', 5)[1].split('.', 5)[0])
test2['Title'] = train2['Name'].apply(lambda x: x.split(', ', 5)[1].split('.', 5)[0])
print(train2['Title'].unique())
print(test2['Title'].unique())

In [ ]:
train2.groupby('Title').Survived.value_counts()

In [ ]:
train2.Title.value_counts()

If were a Miss, Mrs, Master, Sir, Countess, etc. you seem to have had a better chance.

using Cabin data for analysis :----

Maybe we can create variabes using the first letter of the cabin A-F, and include nan as a category, since many passengers didn't have a cabin.

In [ ]:
train.Cabin.unique()

In [ ]:
data = [train2, test2]
import re

for dataset in data:
    dataset['cabin_first_letter'] = dataset['Cabin'].apply(lambda x: str(x)[0])
    dataset['had_cabin'] = [0 if dataset['Cabin'].isnull()[i] else 1 for i in range(len(dataset))]
print(train2['cabin_first_letter'].value_counts())
print(train2['had_cabin'].value_counts())
print(test2['cabin_first_letter'].value_counts())
print(test2['had_cabin'].value_counts())

Examine relationship between cabin and survival.

In [ ]:
train2.groupby(['cabin_first_letter'])['Survived'].mean()

In [ ]:
train2['cabin_first_letter'].value_counts()

In [ ]:
test2['cabin_first_letter'].value_counts()

In [ ]:
train2.groupby('cabin_first_letter')['Survived'].mean().plot(kind='bar', x='cabin_first_letter', y='Survived')
plt.tight_layout()
plt.show()

In [ ]:
for dataset in data:
    dataset['Title'].replace(['Lady', 'Ms', 'Mlle'], 'Miss', inplace=True)
    dataset['Title'].replace('Mme', 'Mrs', inplace=True)

In [ ]:
train2.Title.value_counts()

In [ ]:
test2.Title.value_counts()

In [ ]:
popular_titles = [i for i in train2.Title.unique() if train2.Title.value_counts().loc[i]>10]
print(popular_titles)

In [ ]:
for dataset in data:
    dataset['Popular_title'] = [dataset['Title'][i] if dataset['Title'][i] in popular_titles else 'Other' for i in range(len(dataset))]

In [ ]:
train2['Popular_title'].value_counts()


In [ ]:
test2['Popular_title'].value_counts()

In [ ]:
for dataset in data:
    dataset['Sex'] = [1 if dataset['Sex'][i] == 'male' else 0 for i in range(len(dataset))]

In [ ]:
train2['Embarked'].value_counts()

In [ ]:
train2['Embarked'] = train2['Embarked'].fillna('S')

In [ ]:
train2 = train2.drop(columns=['Fare'])
test2 = test2.drop(columns=['Fare'])

In [ ]:
train2['Embarked'] = train2['Embarked'].map({'S':1, 'C':2, 'Q':3})
test2['Embarked'] = test2['Embarked'].map({'S':1, 'C':2, 'Q':3})

In [ ]:
train2['Popular_title'] = train2['Popular_title'].map({'Mr':1, 'Mrs':2, 'Miss':3, 'Master':4, 'Other':5})
test2['Popular_title'] = test2['Popular_title'].map({'Mr':1, 'Mrs':2, 'Miss':3, 'Master':4, 'Other':5})

In [ ]:
train3 = train2[['Popular_title','Age','Sex','Pclass','log_fare','Parch','SibSp','had_cabin','Survived']]
test3 = test2[['Popular_title','Age','Sex','Pclass','log_fare','Parch','SibSp','had_cabin']]

In [ ]:
train3['fam_size'] = train3['Parch'] + train3['SibSp']
test3['fam_size'] = test3['Parch'] + test3['SibSp']


In [ ]:
train3.drop(columns=['Parch','SibSp'], inplace=True)

In [ ]:
test3.drop(columns=['Parch','SibSp'], inplace=True)

In [ ]:
train3.head()

In [ ]:
test3.tail()

Finally ,**Modeling**  

using ,a Random Forest Classifier, based on the features we think have the greatest impact on survival.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

y = train3["Survived"]

features = ['Age','Sex','Pclass','log_fare','had_cabin']
X = train3[features]

model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
model.fit(X, y)

In [ ]:
X_test = test3[features]

In [ ]:
predictions = model.predict(X_test)

In [ ]:
output = pd.DataFrame({'PassengerId': test2.PassengerId, 'Survived': predictions})
output.to_csv('random_forest.csv', index=False)
print("Your submission was successfully saved! This model was pretty accurate, with a score of " + str(model.score(X,y)))

In [ ]:
features


In [ ]:
model.feature_importances_